In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from ames_preprocessing import restrict_col_list, get_compressed_ames, transformed_df
from ames_model_helper import * 

In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV, RandomizedSearchCV

In [3]:
data = pd.read_csv('data/Ames_Housing_Price_Data.csv', index_col = 0).drop_duplicates().reset_index(drop = True)

## Manual adjustments

In [4]:
housing = data.copy()

#Do not appear to be legit garages; remove
housing.at[531,'GarageType'] = np.nan
housing.at[531,'GarageCars'] = np.nan
housing.at[531,'GarageArea'] = 0
housing.at[433, 'GarageType'] = np.nan

#Fill using known basement finish type
housing.at[2433, 'BsmtFinType2'] = housing.at[2433, 'BsmtFinType1']

#Fill missing exposure & electrical with most frequently occuring
housing.at[813, 'BsmtExposure'] = 'No'
housing.at[1201, 'BsmtExposure'] = 'No'
housing.at[2441, 'Electrical'] = 'SBrkr'

#Fix remodel year which makes no sense
housing.at[2033, 'YearRemodAdd'] = housing.at[2033, 'YearBuilt']

## Create cleaned/compressed dataset & feature lists

In [5]:
data_dict = get_compressed_ames(housing)

In [6]:
housing = data_dict['housing']
areas = data_dict['areas']
frontage = data_dict['frontage']
miscval = data_dict['miscval']
conditions = data_dict['conditions']
inspect10pt = data_dict['inspect10pt']
inspect5pt = data_dict['inspect5pt']
inspections = data_dict['inspections']
dates = data_dict['dates']
counts = data_dict['counts']
other_cats = data_dict['categoricals']

In [7]:
housing['remod_age'] = housing['YrSold'] - housing['YearRemodAdd']
housing.drop('YearRemodAdd', axis = 1, inplace = True)
dates = restrict_col_list(dates, housing)
dates.append('remod_age')

In [8]:
categoricals = other_cats+conditions+inspections
numeric = [x for x in housing.columns if x not in categoricals+['PID', 'SalePrice']]
assert set(housing[numeric].select_dtypes(include=np.number).columns) == set(numeric)

# Create training and test sets

In [9]:
features = housing.drop(['PID', 'SalePrice'], axis = 1)
target = housing.SalePrice

#Create train/test indices
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = .3, random_state = 11)
train_idx = X_train.index
test_idx = X_test.index

# Dummifying

In [10]:
ohe = ColumnTransformer([
    #pass a list of column names which you want to pass through unaltered
    #The second coordinate is usually a transformer, but the keyword 'passthrough'
    #Just passes those columns instead.
    ('pass', 'passthrough', numeric),
    
    #pass a list of columns which you want to be OHE
    ('ohe', OneHotEncoder(drop = 'first', sparse = False), categoricals)
])

features_dummied = transformed_df(ohe,features)

X_train_dum = features_dummied.loc[train_idx,:]
X_test_dum = features_dummied.loc[test_idx,:]

# Tune Lasso - All features

## Scaling all features

In [11]:
# pipe = Pipeline([
#     ('standardize', StandardScaler()),
#     ('lasso', Lasso(tol = .04))
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha': 10.0 ** np.arange(-5,5)}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [12]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [13]:
# pipe = Pipeline([
#     ('standardize', StandardScaler()),
#     ('lasso', Lasso())
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha':  np.arange(0,10000,1000)[1:]}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [14]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [15]:
# pipe = Pipeline([
#     ('standardize', StandardScaler()),
#     ('lasso', Lasso())
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha':  np.arange(0,2000,100)[1:]}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [16]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [17]:
# pipe = Pipeline([
#     ('standardize', StandardScaler()),
#     ('lasso', Lasso())
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha':  np.arange(500,700,10)}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [18]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [19]:
pipe = Pipeline([
    ('standardize', StandardScaler()),
    ('lasso', Lasso())
])
gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha':  np.arange(530,550)}, cv = 5, n_jobs = -1)
gslasso.fit(X_train_dum,y_train)
best_lasso = gslasso.best_estimator_

In [20]:
print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

The best CV score was 0.8823509233540285 using parameters {'lasso__alpha': 546}


The best CV score was 0.8823509233540285 using parameters {'lasso__alpha': 546}

### Evaluate

In [21]:
best_lasso.fit(X_train_dum,y_train)
print('Train score: %s' %best_lasso.score(X_train_dum,y_train))
print('Test score: %s' %best_lasso.score(X_test_dum,y_test))

Train score: 0.9245081312145583
Test score: 0.9184912945610205


In [22]:
print('Average validation score: %s' \
      % np.mean(cross_val_score(best_lasso, X_train_dum, y_train, 
                                cv = 5)))

Average validation score: 0.8823509233540285


Does marginally better than interpretable model.

## Scaling numerics only

In [23]:
scaler = ColumnTransformer([
    ('standardizer', StandardScaler(), numeric),
    ('dummies', 'passthrough', [col for col in features_dummied if col not in numeric])
])

pipe = Pipeline([
    ('standardize', scaler),
    ('lasso', Lasso(tol = .02))
])

np.mean(cross_val_score(pipe, X_train_dum, y_train, cv = 5))

0.8747589330330641

In [24]:
# pipe = Pipeline([
#     ('standardize', scaler),
#     ('lasso', Lasso(tol = .04))
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha': 10.0 ** np.arange(-5,5)}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [25]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [26]:
# pipe = Pipeline([
#     ('standardize', scaler),
#     ('lasso', Lasso())
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha': np.arange(0,2000,100)[1:]}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [27]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [28]:
# pipe = Pipeline([
#     ('standardize', scaler),
#     ('lasso', Lasso())
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha': np.arange(0,200,10)[1:]}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [29]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [30]:
# pipe = Pipeline([
#     ('standardize', scaler),
#     ('lasso', Lasso())
# ])
# gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha': np.arange(90,110)}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [31]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [32]:
pipe = Pipeline([
    ('standardize', scaler),
    ('lasso', Lasso())
])
gslasso = GridSearchCV(pipe, param_grid = {'lasso__alpha': np.arange(103,104, .1)}, cv = 5, n_jobs = -1)
gslasso.fit(X_train_dum,y_train)
best_lasso = gslasso.best_estimator_

In [33]:
print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

The best CV score was 0.8873808435652254 using parameters {'lasso__alpha': 103.79999999999995}


The best CV score was 0.8873808435652254 using parameters {'lasso__alpha': 103.79999999999995}

### Evaluate model

In [34]:
best_lasso.fit(X_train_dum,y_train)
print('Train score: %s' %best_lasso.score(X_train_dum,y_train))
print('Test score: %s' %best_lasso.score(X_test_dum,y_test))

Train score: 0.9197361493180649
Test score: 0.920766568744341


In [35]:
print('Average validation score: %s' \
      % np.mean(cross_val_score(best_lasso, X_train_dum, y_train, 
                                cv = 5)))

Average validation score: 0.8873808435652254


Shows improvement, but still capping out around 92%

## No scaling

In [36]:
# gslasso = GridSearchCV(Lasso(tol = .04), param_grid = {'alpha': 10.0 ** np.arange(-5,5)}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [37]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [38]:
# gslasso = GridSearchCV(Lasso(), param_grid = {'alpha':np.arange(0,2000,100)[1:]}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [39]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [40]:
# gslasso = GridSearchCV(Lasso(tol = .03), param_grid = {'alpha':np.arange(0,200,10)[1:]}, cv = 5, n_jobs = -1)
# gslasso.fit(X_train_dum,y_train)
# best_lasso = gslasso.best_estimator_

In [41]:
# print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

In [42]:
gslasso = GridSearchCV(Lasso(), param_grid = {'alpha':np.arange(90,110)}, cv = 5, n_jobs = -1)
gslasso.fit(X_train_dum,y_train)
best_lasso = gslasso.best_estimator_

In [43]:
print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

The best CV score was 0.8871103967802345 using parameters {'alpha': 100}


The best CV score was 0.8871103967802345 using parameters {'alpha': 100}

### Evaluate model

In [44]:
best_lasso.fit(X_train_dum,y_train)
print('Train score: %s' %best_lasso.score(X_train_dum,y_train))
print('Test score: %s' %best_lasso.score(X_test_dum,y_test))

Train score: 0.9203862715034566
Test score: 0.922006216515469


In [45]:
print('Average validation score: %s' \
      % np.mean(cross_val_score(best_lasso, X_train_dum, y_train, 
                                cv = 5)))

Average validation score: 0.8871103967802345


Has the best performance, confidently around 92%.

# On selected features

The research using selection by lasso, linear independence, and error helped identify features which were better linear predictors. Try an unscaled lasso model on these features alone.

In [46]:
with open('drop_list.txt') as f:
    drop_list = f.read().split(',')

In [47]:
features_select = features_dummied.drop(drop_list,axis = 1)

X_select = features_select.loc[train_idx,:]
X_select_test = features_select.loc[test_idx,:]

In [48]:
gslasso = GridSearchCV(Lasso(), param_grid = {'alpha': 10.0 ** np.arange(-5,5)}, cv = 5, n_jobs = -1)
gslasso.fit(X_select,y_train)
best_lasso = gslasso.best_estimator_

In [49]:
print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

The best CV score was 0.9026388746538331 using parameters {'alpha': 1e-05}


In [50]:
gslasso = GridSearchCV(Lasso(tol = .03), param_grid = {'alpha': 10.0 ** np.arange(-10,-5)}, cv = 5, n_jobs = -1)
gslasso.fit(X_select,y_train)
best_lasso = gslasso.best_estimator_

In [51]:
print(f'The best CV score was {gslasso.best_score_} using parameters {gslasso.best_params_}')

The best CV score was 0.9026388747855361 using parameters {'alpha': 1e-10}


Once features have been subsetted, lasso turns into unpenalized regression, with alpha becoming very small.

### Evaluate model

In [52]:
best_lasso.fit(X_train_dum,y_train)
print('Train score: %s' %best_lasso.score(X_train_dum,y_train))
print('Test score: %s' %best_lasso.score(X_test_dum,y_test))

Train score: 0.9323821191601597
Test score: 0.9205159082127955


/Users/zachstone/miniforge3/envs/env_tf/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.293e+11, tolerance: 2.922e+11
  model = cd_fast.enet_coordinate_descent(


Again, raises test score to 92% (slightly overfit), showing that interpretable model is competitive.